Use cellchat to classify samples

In [1]:
suppressPackageStartupMessages({
    library(CellChat, quietly = T)
    library(patchwork, quietly = T)
    library(RhpcBLASctl, quietly = T)
    library(Matrix, quietly = T)
    library(rhdf5, quietly = T)
    library(stringr, quietly = T)
})
options(stringsAsFactors = FALSE)
# expression_data_path = '/data2/hratch/immune_CCI/covid/covid_atlas/interim/umi_for_timing/'
input_path = '/data2/hratch/immune_CCI/covid/covid_atlas/interim/classification_inputs/'

# RhpcBLASctl::blas_set_num_threads(20) 

In [2]:
# parameters
cell_grouper<-'majorType'
seed = 888
# inputs
group = FALSE
sample.names<-c('S-S083', 'S-M001', 'S-HC018-1')

Load data:

In [3]:
# load metadata
md.cell <- read.csv(paste0(input_path,'metadata.csv'), row.names = 1)

sample_names<-unique(md.cell$sampleID)
sample_context_map<-list()
for (sn in sample_names){
    context<-unique(md.cell[md.cell$sampleID == sn,'CoVID.19.severity'])
    if (length(context) != 1){stop('Incorred sample to context mapping')}
    sample_context_map[[sn]]<-context
}
contexts<-unique(sample_context_map)
# load LR pairs
# filter for the LR pairs used by Tensor cell2cell
# lr_pairs<-read.csv(paste0(input_data_path,'Tensor-cell2cell-LRpairs.csv'))
# lr_pairs<-lr_pairs$interaction_name
humandb<-CellChatDB.human
# humandb$interaction<-CellChatDB.human$interaction[CellChatDB.human$interaction$interaction_name %in% lr_pairs, ] 
# saveRDS(humandb, paste0(output_results_path, 'humandb.rds'))

In [4]:
# load the UMI counts
read_sample_h5<-function(sn){
    counts<-h5read(paste0(input_path, 'umi_per_sample.h5'), sn)
    count<-counts[[4]]
    colnames(count)<-counts[[2]]
    rownames(count)<-sapply(counts[[1]], function(x) str_replace_all(x, '-', '.')) 
    return(count)
}


if (!group){
    counts<-lapply(setNames(sample.names, sample.names), function(sn) read_sample_h5(sn))
}else{ # group by context
    by.context<-lapply(setNames(contexts, contexts), function(context) names(sample_context_map[sample.names][sample_context_map[sample.names] == context]))
    
    group_by_context<-function(context){
        sns<-by.context[[context]]       
        counts<-lapply(sns, function(sn) read_sample_h5(sn))    
        counts<-do.call(cbind, counts)
        return (counts)
    }
    counts<-lapply(setNames(contexts, contexts), function(context) group_by_context(context))                   
                    
}

Run cellchat:

In [5]:
# create cellchat object for each sample or sample.name
covid.list<-list()
for (sample.name in names(counts)){
    # loop through each sample.name and create a cell type future
    expr<-CellChat::normalizeData(counts[[sample.name]])
    cellchat<-createCellChat(object = as(expr, "dgCMatrix"), meta = md.cell[colnames(expr),], 
                                   group.by = cell_grouper)
    cellchat@DB <- humandb # human organism

    cellchat <- subsetData(cellchat) # subset the expression data of signaling genes, assign to @data.signalling 
    cellchat <- identifyOverExpressedGenes(cellchat)
    cellchat <- identifyOverExpressedInteractions(cellchat) # generate @ LR slot used by computeCommunProb
    cellchat <- projectData(cellchat, PPI.human) # shallow sequencing depth
    
    cellchat <- computeCommunProb(cellchat, raw.use = F, type = 'triMean', trim = NULL, seed.use = seed, 
                                 population.size = F) 
    
    # The functional similarity analysis requires the same cell population composition between two datasets.
    cellchat <- filterCommunication(cellchat, min.cells = 10)
    cellchat <- computeCommunProbPathway(cellchat)
    covid.list[[sample.name]]<-cellchat
}

# merge and analyze
cellchat <- mergeCellChat(covid.list, add.names = names(covid.list))
cellchat <- computeNetSimilarityPairwise(cellchat, type = 'structural')
cellchat.embed <- netEmbedding(cellchat, type = 'structural')   

Create a CellChat object from a data matrix

Set cell identities for the new CellChat object



The cell groups used for CellChat analysis are  B CD4 CD8 DC Macro Mega Mono Neu NK Plasma 
The cell-cell communication related with the following cell groups are excluded due to the few number of cells:  Macro Neu Plasma 


Create a CellChat object from a data matrix

Set cell identities for the new CellChat object



The cell groups used for CellChat analysis are  B CD4 CD8 DC Mega Mono NK Plasma 
The cell-cell communication related with the following cell groups are excluded due to the few number of cells:  CD4 Mega Plasma 


Create a CellChat object from a data matrix

Set cell identities for the new CellChat object



The cell groups used for CellChat analysis are  B CD4 CD8 DC Mega Mono NK Plasma 
The cell-cell communication related with the following cell groups are excluded due to the few number of cells:  DC Mega 


Merge the following slots: 'data.signaling','net', 'netP','meta', 'idents', 'var.features' , 'DB', and 'LR'.



Compute signaling network similarity for datasets 1 2 3 
Manifold learning of the signaling networks for datasets 1 2 3 


Begin classification:

In [12]:
head(cellchat.embed@netP$similarity)

,TGFb--S-S083,GDNF--S-S083,ACTIVIN--S-S083,EGF--S-S083,NRG--S-S083,PDGF--S-S083,VEGF--S-S083,IGF--S-S083,CCL--S-S083,CXCL--S-S083,⋯,LCK--S-HC018-1,MPZ--S-HC018-1,NCAM--S-HC018-1,NOTCH--S-HC018-1,PECAM1--S-HC018-1,PVR--S-HC018-1,SELPLG--S-HC018-1,SEMA4--S-HC018-1,SEMA7--S-HC018-1,TIGIT--S-HC018-1
TGFb--S-S083,1.0000000,0.00000000,0.00000000,0.22159293,0.3766344,0.00000000,0.5131542,0.8715016,0.00000000,0.0000000,⋯,0.0000000,0.09683851,0.0000000,0.2194184,0.00000000,0.00000000,0.00000000,0.0000000,0.0000000,0.00000000
GDNF--S-S083,0.0000000,1.00000000,0.38191270,0.00000000,0.0000000,0.32498550,0.0000000,0.0000000,0.00000000,0.0000000,⋯,0.0000000,0.00000000,0.0000000,0.0000000,0.00000000,0.00000000,0.00000000,0.0000000,0.0000000,0.00000000
ACTIVIN--S-S083,0.0000000,0.38191270,1.00000000,0.00000000,0.0000000,0.88075522,0.0000000,0.0000000,0.00000000,0.0000000,⋯,0.0000000,0.00000000,0.0000000,0.0000000,0.00000000,0.00000000,0.00000000,0.0000000,0.0000000,0.00000000
EGF--S-S083,0.2215929,0.00000000,0.00000000,1.00000000,0.0000000,0.00000000,0.4441580,0.1755937,0.06264874,0.0000000,⋯,0.0000000,0.00000000,0.0000000,0.0000000,0.00000000,0.00000000,0.06225686,0.0000000,0.0000000,0.00000000
NRG--S-S083,0.3766344,0.00000000,0.00000000,0.00000000,1.0000000,0.00000000,0.1348145,0.4366473,0.00000000,0.0000000,⋯,0.0000000,0.37395470,0.0000000,0.4988881,0.00000000,0.00000000,0.00000000,0.0000000,0.0000000,0.00000000
PDGF--S-S083,0.0000000,0.32498550,0.88075522,0.00000000,0.0000000,1.00000000,0.0000000,0.0000000,0.00000000,0.0000000,⋯,0.0000000,0.00000000,0.0000000,0.0000000,0.00000000,0.00000000,0.00000000,0.0000000,0.0000000,0.00000000
VEGF--S-S083,0.5131542,0.00000000,0.00000000,0.44415803,0.1348145,0.00000000,1.0000000,0.4399044,0.00000000,0.0000000,⋯,0.0000000,0.00000000,0.0000000,0.1742170,0.00000000,0.00000000,0.00000000,0.0000000,0.0000000,0.00000000
IGF--S-S083,0.8715016,0.00000000,0.00000000,0.17559367,0.4366473,0.00000000,0.4399044,1.0000000,0.00000000,0.0000000,⋯,0.0000000,0.09672029,0.0000000,0.2201007,0.00000000,0.00000000,0.00000000,0.0000000,0.0000000,0.00000000
CCL--S-S083,0.0000000,0.00000000,0.00000000,0.06264874,0.0000000,0.00000000,0.0000000,0.0000000,1.00000000,0.0000000,⋯,0.0000000,0.00000000,0.0000000,0.0000000,0.00000000,0.00000000,0.74085788,0.2481117,0.0000000,0.00000000
CXCL--S-S083,0.0000000,0.00000000,0.00000000,0.00000000,0.0000000,0.00000000,0.0000000,0.0000000,0.00000000,1.0000000,⋯,0.5667825,0.00000000,0.0000000,0.0000000,0.00000000,0.00000000,0.00000000,0.0000000,0.5668360,0.00000000


In [8]:
cellchat@options

An object of class CellChat created from a merged object with multiple datasets 
 919 signaling genes.
 10573 cells.